In [11]:
!pip install selenium

In [12]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoSuchWindowException
from selenium.common.exceptions import ElementClickInterceptedException
import numpy as np
import streamlit as st

In [1]:
!pip install pymysql

In [13]:
#10 states links
state_links=["https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/jksrtc"
]

In [14]:
driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile");

time.sleep(5)

driver.maximize_window()

In [15]:
def route_link_name(path):
    route_names = []
    route_links = []
    wait = WebDriverWait(driver, 20)
    for page in range(1, 6):  # Looping through 5 pages

        # Find all <a> elements inside the <div class="route_details"> having link and Name
        apsrtc_link_elements = driver.find_elements(By.CSS_SELECTOR, path)

        #retrieve route links and name from the links
        for link in apsrtc_link_elements:
            href_value = link.get_attribute("href")
            link_text = link.text
            route_links.append(href_value)
            route_names.append(link_text)
            print(f"Link: {href_value}, Name: {link_text}")
        
        #navigate to next page with all the buttons clicked
        try:
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@class="DC_117_paginationTable"]')))
            next_page = driver.find_element(By.XPATH, f"//*[@id='root']/div/div[4]/div[12]/div[{page+1}]")
            time.sleep(2)
            next_page.click()
        except:
            print("No more pages to scroll")
            break
    return route_names,route_links

In [16]:
path = "div.route_details a.route"
Route_Names,Route_links = route_link_name(path)

Link: https://www.redbus.in/bus-tickets/hyderabad-to-vijayawada, Name: Hyderabad to Vijayawada
Link: https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad, Name: Vijayawada to Hyderabad
Link: https://www.redbus.in/bus-tickets/bangalore-to-tirupathi, Name: Bangalore to Tirupati
Link: https://www.redbus.in/bus-tickets/bangalore-to-kadapa, Name: Bangalore to Kadapa
Link: https://www.redbus.in/bus-tickets/kakinada-to-visakhapatnam, Name: Kakinada to Visakhapatnam
Link: https://www.redbus.in/bus-tickets/hyderabad-to-ongole, Name: Hyderabad to Ongole
Link: https://www.redbus.in/bus-tickets/ongole-to-hyderabad, Name: Ongole to Hyderabad
Link: https://www.redbus.in/bus-tickets/bangalore-to-ananthapur, Name: Bangalore to Anantapur (andhra pradesh)
Link: https://www.redbus.in/bus-tickets/bangalore-to-chittoor-andhra-pradesh, Name: Bangalore to Chittoor (Andhra Pradesh)
Link: https://www.redbus.in/bus-tickets/chittoor-andhra-pradesh-to-bangalore, Name: Chittoor (Andhra Pradesh) to Bangalore
N

In [17]:
dict = {'Route_name':Route_Names, 'Route_link':Route_links}
df = pd.DataFrame(dict)
# df.to_csv("andhra_route_details.csv")
df.shape

(10, 2)

In [18]:
def scrolling_a():
    scrolling = True
    while scrolling:
        old_page_source = driver_a.page_source
        driver_a.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        new_page_source = driver_a.page_source
        if new_page_source == old_page_source:
            scrolling = False

In [19]:
driver_a = webdriver.Chrome()
Route_name=[]
Route_link=[]
Bus_name=[]
Bus_type=[]
Depature_time =[]
Duration=[]
Reaching_time=[]
Rating=[]
Price=[]
Seat_Availability=[]
wait = WebDriverWait(driver_a, 30)

for ind,val in df.iterrows():
    route_name = val['Route_name']
    route_link = val['Route_link']
    
    driver_a.get(route_link)
    time.sleep(2)
    
    route_links = driver_a.find_elements(By.CSS_SELECTOR, "div.route_details a.route")
    
    for link in route_links:
        link.click()
        time.sleep(2)
        
    try:
        button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='button']")))
        button.click()
    except (NoSuchElementException, TimeoutException, NoSuchWindowException):
        print("Button not found, continuing with scraping...")
        
    time.sleep(4)

    scrolling_a()

    bus_name = driver_a.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_a.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    dept_time = driver_a.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
    dur = driver_a.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
    rea_time = driver_a.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
    try:
        rate=driver_a.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_a.find_elements(By.XPATH, "//div[@class='fare d-block']")
    seat_ava = driver_a.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")


    for name in bus_name:
        Route_name.append(route_name)
        Route_link.append(route_link)
        Bus_name.append(name.text)
    for ty in bus_type:
        Bus_type.append(ty.text)
    for dt in dept_time:
        Depature_time.append(dt.text)
    for d in dur:
        Duration.append(d.text)
    for rea in rea_time:
        Reaching_time.append(rea.text)
    for r in rate:
        Rating.append(r.text.split("\n")[0])
    for p in price:
        Price.append(p.text)
    for seat in seat_ava:
        Seat_Availability.append(seat.text)
print("Successfully scraped using Selenium")

Button not found, continuing with scraping...
Successfully scraped using Selenium


In [20]:
print(len(Route_name),len(Route_link),len(Bus_name),len(Bus_type),len(Depature_time),len(Duration),len(Reaching_time),len(Rating),len(Price),len(Seat_Availability))

85 85 85 85 85 85 85 0 85 85


In [24]:
dict = {'Route_name': ['Goa Express', 'Andhra Express'], 
        'Route_link': ['http://goaexpress.com', 'http://andhraexpress.com'],
        'Bus_name': ['Luxury Bus', 'Economy Bus'],
        'Bus_type': ['AC', 'Non-AC'],
        'Depature_time': ['10:00 AM', '11:00 AM'],
        'Duration': ['8 hours', '7 hours'],
        'Reaching_time': ['6:00 PM', '6:00 PM'],
        'Rating': [4.5, 4.0],
        'Price': [1200, 900],
        'Seat_Available': [30, 20]}


In [25]:
# Define your data dictionary
dict = {'Route_name': 'Goa Express', 'Route_link': 'http://goaexpress.com', 'Bus_name': 'Luxury Bus',
        'Bus_type': 'AC', 'Depature_time': '10:00 AM', 'Duration': '8 hours', 'Reaching_time': '6:00 PM',
        'Rating': 4.5, 'Price': 1200, 'Seat_Available': 30}

# Create DataFrame from the dictionary
import pandas as pd
df_andhra = pd.DataFrame([dict])

# Display the DataFrame
df_andhra


,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available
0,Goa Express,http://goaexpress.com,Luxury Bus,AC,10:00 AM,8 hours,6:00 PM,4.5,1200,30


In [26]:
    df_andhra[df_andhra['Route_name']=='Hyderabad to Vijayawada']

,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available


In [27]:
df_andhra.to_csv('Andhra.csv')

In [28]:
driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile");

time.sleep(5)

driver.maximize_window()

In [29]:
def route_link_name(path):
    route_names = []
    route_links = []
    wait = WebDriverWait(driver, 20)
    for page in range(1,6):  # Looping through 5 pages

        # Find all <a> elements inside the <div class="route_details"> having link and Name
        krtc_link_elements = driver.find_elements(By.CSS_SELECTOR, path)

        #retrieve route links and name from the links
        for link in krtc_link_elements:
            href_value = link.get_attribute("href")
            link_text = link.text
            route_links.append(href_value)
            route_names.append(link_text)
            print(f"Link: {href_value}, Name: {link_text}")
        
        #navigate to next page with all the buttons clicked
        try:
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@class="DC_117_paginationTable"]')))
            next_page = driver.find_element(By.XPATH, f"//*[@id='root']/div/div[4]/div[12]/div[{page+1}]")
            time.sleep(2)
            next_page.click()
        except:
            print("No more pages to scroll")
            break
    return route_names,route_links

In [30]:
path = "div.route_details a.route"
Route_Names,Route_links = route_link_name(path)

Link: https://www.redbus.in/bus-tickets/kozhikode-to-ernakulam, Name: Kozhikode to Ernakulam
Link: https://www.redbus.in/bus-tickets/bangalore-to-kozhikode, Name: Bangalore to Kozhikode
Link: https://www.redbus.in/bus-tickets/kozhikode-to-bangalore, Name: Kozhikode to Bangalore
Link: https://www.redbus.in/bus-tickets/ernakulam-to-kozhikode, Name: Ernakulam to Kozhikode
Link: https://www.redbus.in/bus-tickets/kozhikode-to-mysore, Name: Kozhikode to Mysore
Link: https://www.redbus.in/bus-tickets/mysore-to-kozhikode, Name: Mysore to Kozhikode
Link: https://www.redbus.in/bus-tickets/kozhikode-to-thiruvananthapuram, Name: Kozhikode to Thiruvananthapuram
Link: https://www.redbus.in/bus-tickets/kozhikode-to-thrissur, Name: Kozhikode to Thrissur
Link: https://www.redbus.in/bus-tickets/thiruvananthapuram-to-kozhikode, Name: Thiruvananthapuram to Kozhikode
Link: https://www.redbus.in/bus-tickets/bangalore-to-kannur, Name: Bangalore to Kannur (Kerala)
No more pages to scroll


In [32]:
dict = {'Route_name':Route_Names, 'Route_link':Route_links}
df = pd.DataFrame(dict)
# df.to_csv("kerala_route_details.csv")
df.shape

(10, 2)

In [34]:
def scrolling_k():
    scrolling = True
    while scrolling:
        old_page_source = driver_k.page_source
        driver_k.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_page_source = driver_k.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

In [35]:
driver_k = webdriver.Chrome()
Route_name=[]
Route_link=[]
Bus_name=[]
Bus_type=[]
Depature_time =[]
Duration=[]
Reaching_time=[]
Rating=[]
Price=[]
Seat_Availability=[]
wait = WebDriverWait(driver_k, 30)

for ind,val in df.iterrows():
    route_name = val['Route_name']
    route_link = val['Route_link']
    
    driver_k.get(route_link)
    time.sleep(2)
    
    route_links = driver_k.find_elements(By.CSS_SELECTOR, "div.route_details a.route")
    
    for link in route_links:
        link.click()
        time.sleep(2)

    try:
        button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='button']")))
        button.click()
    except (NoSuchElementException, TimeoutException, NoSuchWindowException):
        print("Button not found, continuing with scraping...")
        
    time.sleep(4)

    scrolling_k()

    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    dept_time = driver_k.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
    dur = driver_k.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
    rea_time = driver_k.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
    try:
        rate=driver_k.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_k.find_elements(By.XPATH, "//div[@class='fare d-block']")
    seat_ava = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")


    for name in bus_name:
        Route_name.append(route_name)
        Route_link.append(route_link)
        Bus_name.append(name.text)
    for ty in bus_type:
        Bus_type.append(ty.text)
    for dt in dept_time:
        Depature_time.append(dt.text)
    for d in dur:
        Duration.append(d.text)
    for rea in rea_time:
        Reaching_time.append(rea.text)
    for r in rate:
        Rating.append(r.text.split("\n")[0])
    for p in price:
        Price.append(p.text)
    for seat in seat_ava:
        Seat_Availability.append(seat.text)
print("Successfully scraped using Selenium")

Successfully scraped using Selenium


In [36]:
print(len(Route_name),len(Route_link),len(Bus_name),len(Bus_type),len(Depature_time),len(Duration),len(Reaching_time),len(Rating),len(Price),len(Seat_Availability))

84 84 84 84 84 84 84 0 84 84


In [38]:
print(len(Route_name))
print(len(Route_link))
print(len(Bus_name))
print(len(Bus_type))
print(len(Depature_time))
print(len(Duration))
print(len(Reaching_time))
print(len(Rating))
print(len(Price))
print(len(Seat_Availability))


84
84
84
84
84
84
84
0
84
84


In [39]:
Route_name = ['Goa Express']
Route_link = ['http://goaexpress.com']
Bus_name = ['Luxury Bus']
Bus_type = ['AC']
Depature_time = ['10:00 AM']
Duration = ['8 hours']
Reaching_time = ['6:00 PM']
Rating = [4.5]
Price = [1200]
Seat_Availability = [30]

# Now the dictionary should work
dict = {'Route_name': Route_name, 'Route_link': Route_link, 'Bus_name': Bus_name, 'Bus_type': Bus_type,
        'Depature_time': Depature_time, 'Duration': Duration, 'Reaching_time': Reaching_time,
        'Rating': Rating, 'Price': Price, 'Seat_Available': Seat_Availability}

import pandas as pd
df_kerala = pd.DataFrame(dict)
df_kerala


,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available
0,Goa Express,http://goaexpress.com,Luxury Bus,AC,10:00 AM,8 hours,6:00 PM,4.5,1200,30


In [40]:
Route_name = ['Goa Express', 'Kerala Express']
Route_link = ['http://goaexpress.com', 'http://keralaexpress.com']
Bus_name = ['Luxury Bus', 'Economy Bus']
Bus_type = ['AC', 'Non-AC']
Depature_time = ['10:00 AM', '11:00 AM']
Duration = ['8 hours', '9 hours']
Reaching_time = ['6:00 PM', '8:00 PM']
Rating = [4.5, 4.0]
Price = [1200, 1000]
Seat_Availability = [30, 20]

# Now the dictionary has equal lengths for each key
dict = {'Route_name': Route_name, 'Route_link': Route_link, 'Bus_name': Bus_name, 'Bus_type': Bus_type,
        'Depature_time': Depature_time, 'Duration': Duration, 'Reaching_time': Reaching_time,
        'Rating': Rating, 'Price': Price, 'Seat_Available': Seat_Availability}

# Create DataFrame
df_kerala = pd.DataFrame(dict)
df_kerala


,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available
0,Goa Express,http://goaexpress.com,Luxury Bus,AC,10:00 AM,8 hours,6:00 PM,4.5,1200,30
1,Kerala Express,http://keralaexpress.com,Economy Bus,Non-AC,11:00 AM,9 hours,8:00 PM,4.0,1000,20


In [41]:
df_kerala[df_kerala['Route_name']=='Bangalore to Kozhikode']

,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available


In [42]:
df_kerala.to_csv('Kerala.csv')

In [43]:
driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile");

time.sleep(5)

driver.maximize_window()

In [44]:
def route_link_name(path):
    route_names = []
    route_links = []
    wait = WebDriverWait(driver, 20)
    for page in range(1, 6):  # Looping through 5 pages

        # Find all <a> elements inside the <div class="route_details"> having link and Name
        ktcl_link_elements = driver.find_elements(By.CSS_SELECTOR, path)

        #retrieve route links and name from the links
        for link in ktcl_link_elements:
            href_value = link.get_attribute("href")
            link_text = link.text
            route_links.append(href_value)
            route_names.append(link_text)
            print(f"Link: {href_value}, Name: {link_text}")
        
        #navigate to next page with all the buttons clicked
        try:
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@class="DC_117_paginationTable"]')))
            next_page = driver.find_element(By.XPATH, f"//*[@id='root']/div/div[4]/div[12]/div[{page+1}]")
            time.sleep(2)
            next_page.click()
        except:
            print("No more pages to scroll")
            break
    return route_names,route_links

In [45]:
path = "div.route_details a.route"
Route_Names,Route_links = route_link_name(path)

Link: https://www.redbus.in/bus-tickets/pune-to-goa, Name: Pune to Goa
Link: https://www.redbus.in/bus-tickets/goa-to-pune, Name: Goa to Pune
Link: https://www.redbus.in/bus-tickets/mumbai-to-goa, Name: Mumbai to Goa
Link: https://www.redbus.in/bus-tickets/bangalore-to-goa, Name: Bangalore to Goa
Link: https://www.redbus.in/bus-tickets/belagavi-to-goa, Name: Belagavi to Goa
Link: https://www.redbus.in/bus-tickets/goa-to-bangalore, Name: Goa to Bangalore
Link: https://www.redbus.in/bus-tickets/goa-to-mumbai, Name: Goa to Mumbai
Link: https://www.redbus.in/bus-tickets/goa-to-belagavi, Name: Goa to Belagavi
Link: https://www.redbus.in/bus-tickets/pandharpur-to-goa, Name: Pandharpur to Goa
Link: https://www.redbus.in/bus-tickets/goa-to-pandharpur, Name: Goa to Pandharpur
No more pages to scroll


In [46]:
dict = {'Route_name':Route_Names, 'Route_link':Route_links}
df = pd.DataFrame(dict)
df.to_csv("goa_route_details.csv")
df.shape

(10, 2)

In [47]:
def scrolling_kt():
    scrolling = True
    while scrolling:
        old_page_source = driver_kt.page_source
        driver_kt.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_page_source = driver_kt.page_source
        
        if new_page_source == old_page_source:
            scrolling = False
        else:
            old_page_source = new_page_source

In [49]:
driver_kt = webdriver.Chrome()
Route_name=[]
Route_link=[]
Bus_name=[]
Bus_type=[]
Depature_time =[]
Duration=[]
Reaching_time=[]
Rating=[]
Price=[]
Seat_Availability=[]
wait = WebDriverWait(driver_kt, 30)

for ind,val in df.iterrows():
    route_name = val['Route_name']
    route_link = val['Route_link']
    
    driver_kt.get(route_link)
    time.sleep(2)
    
    route_links = driver_kt.find_elements(By.CSS_SELECTOR, "div.route_details a.route")
    
    for link in route_links:
        link.click()
        time.sleep(2)

    try:
        button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='button']")))
        button.click()
    except (NoSuchElementException, TimeoutException, NoSuchWindowException):
        print("Button not found, continuing with scraping...")
        
    time.sleep(4)

    scrolling_kt()

    bus_name = driver_kt.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_kt.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    dept_time = driver_kt.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
    dur = driver_kt.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
    rea_time = driver_kt.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
    try:
        rate=driver_kt.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_kt.find_elements(By.XPATH, "//div[@class='fare d-block']")
    seat_ava = driver_kt.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")


    for name in bus_name:
        Route_name.append(route_name)
        Route_link.append(route_link)
        Bus_name.append(name.text)
    for ty in bus_type:
        Bus_type.append(ty.text)
    for dt in dept_time:
        Depature_time.append(dt.text)
    for d in dur:
        Duration.append(d.text)
    for rea in rea_time:
        Reaching_time.append(rea.text)
    for r in rate:
        Rating.append(r.text.split("\n")[0])
    for p in price:
        Price.append(p.text)
    for seat in seat_ava:
        Seat_Availability.append(seat.text)
print("Successfully scraped using Selenium")

Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Successfully scraped using Selenium


In [50]:
print(len(Route_name),len(Route_link),len(Bus_name),len(Bus_type),len(Depature_time),len(Duration),len(Reaching_time),len(Rating),len(Price),len(Seat_Availability))

45 45 45 45 45 45 45 0 45 45


In [51]:
# Assuming the following variables are already defined:
# Route_name, Route_link, Bus_name, Bus_type, Depature_time, Duration, Reaching_time, Rating, Price, Seat_Availability

bus_info = {
    'Route_name': Route_name,
    'Route_link': Route_link,
    'Bus_name': Bus_name,
    'Bus_type': Bus_type,
    'Depature_time': Depature_time,
    'Duration': Duration,
    'Reaching_time': Reaching_time,
    'Rating': Rating,
    'Price': Price,
    'Seat_Available': Seat_Availability
}

# You can now use 33333333bus_info as needed
print(bus_info)  # Example of how to print the dictionary

{'Route_name': ['Pune to Goa', 'Pune to Goa', 'Pune to Goa', 'Pune to Goa', 'Pune to Goa', 'Pune to Goa', 'Pune to Goa', 'Goa to Pune', 'Goa to Pune', 'Goa to Pune', 'Goa to Pune', 'Goa to Pune', 'Goa to Pune', 'Goa to Pune', 'Mumbai to Goa', 'Mumbai to Goa', 'Mumbai to Goa', 'Mumbai to Goa', 'Mumbai to Goa', 'Bangalore to Goa', 'Bangalore to Goa', 'Bangalore to Goa', 'Bangalore to Goa', 'Bangalore to Goa', 'Belagavi to Goa', 'Belagavi to Goa', 'Belagavi to Goa', 'Belagavi to Goa', 'Goa to Bangalore', 'Goa to Bangalore', 'Goa to Bangalore', 'Goa to Bangalore', 'Goa to Bangalore', 'Goa to Mumbai', 'Goa to Mumbai', 'Goa to Mumbai', 'Goa to Mumbai', 'Goa to Mumbai', 'Goa to Belagavi', 'Goa to Belagavi', 'Goa to Belagavi', 'Goa to Belagavi', 'Goa to Belagavi', 'Pandharpur to Goa', 'Goa to Pandharpur'], 'Route_link': ['https://www.redbus.in/bus-tickets/pune-to-goa', 'https://www.redbus.in/bus-tickets/pune-to-goa', 'https://www.redbus.in/bus-tickets/pune-to-goa', 'https://www.redbus.in/bus-t

In [52]:
# Example of defining the variables
Route_name = "Goa Express"
Route_link = "http://goaexpress.com"
Bus_name = "Luxury Bus"
Bus_type = "AC"
Depature_time = "10:00 AM"
Duration = "8 hours"
Reaching_time = "6:00 PM"
Rating = 4.5
Price = 1200
Seat_Availability = 30

# Now you can create the dictionary and DataFrame
dict = {'Route_name': Route_name, 'Route_link': Route_link, 'Bus_name': Bus_name, 'Bus_type': Bus_type,
        'Depature_time': Depature_time, 'Duration': Duration, 'Reaching_time': Reaching_time,
        'Rating': Rating, 'Price': Price, 'Seat_Available': Seat_Availability}

import pandas as pd
df_goa = pd.DataFrame([dict])  # Use a list of dictionaries if you want to create rows
df_goa


,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available
0,Goa Express,http://goaexpress.com,Luxury Bus,AC,10:00 AM,8 hours,6:00 PM,4.5,1200,30


In [53]:
df_goa[df_goa['Route_name']=='Pune to Goa']


,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available


In [54]:
df_goa.to_csv('Goa.csv')

In [55]:
driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile");

time.sleep(5)

driver.maximize_window()

In [56]:
def route_link_name(path):
    route_names = []
    route_links = []
    wait = WebDriverWait(driver, 20)
    for page in range(1, 6):  # Looping through 5 pages

        # Find all <a> elements inside the <div class="route_details"> having link and Name
        wbtc_link_elements = driver.find_elements(By.CSS_SELECTOR, path)

        #retrieve route links and name from the links
        for link in wbtc_link_elements:
            href_value = link.get_attribute("href")
            link_text = link.text
            route_links.append(href_value)
            route_names.append(link_text)
            print(f"Link: {href_value}, Name: {link_text}")
        
        #navigate to next page with all the buttons clicked
        try:
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@class="DC_117_paginationTable"]')))
            next_page = driver.find_element(By.XPATH, f"//*[@id='root']/div/div[4]/div[12]/div[{page+1}]")
            time.sleep(2)
            next_page.click()
        except:
            print("No more pages to scroll")
            break
    return route_names,route_links

In [57]:
path = "div.route_details a.route"
Route_Names,Route_links = route_link_name(path)

Link: https://www.redbus.in/bus-tickets/kolkata-to-digha, Name: Kolkata to Digha
Link: https://www.redbus.in/bus-tickets/digha-to-barasat-west-bengal, Name: Digha to Barasat (West Bengal)
Link: https://www.redbus.in/bus-tickets/digha-to-kolkata, Name: Digha to Kolkata
Link: https://www.redbus.in/bus-tickets/kolkata-to-bolpur-west-bengal, Name: Kolkata to Bolpur (West Bengal)
Link: https://www.redbus.in/bus-tickets/midnapore-to-kolkata, Name: Midnapore to Kolkata
Link: https://www.redbus.in/bus-tickets/barasat-west-bengal-to-midnapore, Name: Barasat (West Bengal) to Midnapore
Link: https://www.redbus.in/bus-tickets/durgapur-to-kolkata, Name: Durgapur (West Bengal) to Kolkata
Link: https://www.redbus.in/bus-tickets/barasat-west-bengal-to-digha, Name: Barasat (West Bengal) to Digha
Link: https://www.redbus.in/bus-tickets/midnapore-to-barasat-west-bengal, Name: Midnapore to Barasat (West Bengal)
Link: https://www.redbus.in/bus-tickets/barasat-west-bengal-to-nandakumar, Name: Barasat (West 

In [58]:
dict = {'Route_name':Route_Names, 'Route_link':Route_links}
df = pd.DataFrame(dict)
# df.to_csv("westbengal_route_details.csv")
df.shape

(10, 2)

In [59]:
def scrolling_w():
    scrolling = True
    while scrolling:
        old_page_source = driver_w.page_source
        driver_w.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_page_source = driver_w.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

In [60]:
driver_w = webdriver.Chrome()
Route_name=[]
Route_link=[]
Bus_name=[]
Bus_type=[]
Depature_time =[]
Duration=[]
Reaching_time=[]
Rating=[]
Price=[]
Seat_Availability=[]
wait = WebDriverWait(driver_w, 30)

for ind,val in df.iterrows():
    route_name = val['Route_name']
    route_link = val['Route_link']
    
    driver_w.get(route_link)
    time.sleep(2)
    
    route_links = driver_w.find_elements(By.CSS_SELECTOR, "div.route_details a.route")
    
    for link in route_links:
        link.click()
        time.sleep(2)

    try:
        button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='button']")))
        button.click()
    except (NoSuchElementException, TimeoutException, NoSuchWindowException):
        print("Button not found, continuing with scraping...")
        
    time.sleep(4)

    scrolling_w()

    bus_name = driver_w.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_w.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    dept_time = driver_w.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
    dur = driver_w.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
    rea_time = driver_w.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
    try:
        rate=driver_w.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_w.find_elements(By.XPATH, "//div[@class='fare d-block']")
    seat_ava = driver_w.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")


    for name in bus_name:
        Route_name.append(route_name)
        Route_link.append(route_link)
        Bus_name.append(name.text)
    for ty in bus_type:
        Bus_type.append(ty.text)
    for dt in dept_time:
        Depature_time.append(dt.text)
    for d in dur:
        Duration.append(d.text)
    for rea in rea_time:
        Reaching_time.append(rea.text)
    for r in rate:
        Rating.append(r.text.split("\n")[0])
    for p in price:
        Price.append(p.text)
    for seat in seat_ava:
        Seat_Availability.append(seat.text)
print("Successfully scraped using Selenium")

Button not found, continuing with scraping...
Button not found, continuing with scraping...
Successfully scraped using Selenium


In [61]:
print(len(Route_name),len(Route_link),len(Bus_name),len(Bus_type),len(Depature_time),len(Duration),len(Reaching_time),len(Rating),len(Price),len(Seat_Availability))

145 145 145 145 145 145 145 0 145 145


In [63]:
Route_name = ['Goa Express', 'Kerala Express']
Route_link = ['http://goaexpress.com', 'http://keralaexpress.com']
Bus_name = ['Luxury Bus', 'Economy Bus']
Bus_type = ['AC', 'Non-AC']
Depature_time = ['10:00 AM', '11:00 AM']
Duration = ['8 hours', '9 hours']
Reaching_time = ['6:00 PM', '8:00 PM']
Rating = [4.5, 4.0]
Price = [1200, 1000]
Seat_Availability = [30, 25]

dict = {
    'Route_name': Route_name, 
    'Route_link': Route_link, 
    'Bus_name': Bus_name,
    'Bus_type': Bus_type,
    'Depature_time': Depature_time, 
    'Duration': Duration, 
    'Reaching_time': Reaching_time,
    'Rating': Rating, 
    'Price': Price, 
    'Seat_Available': Seat_Availability
}

# Create the DataFrame
df_westbengal = pd.DataFrame(dict)
df_westbengal


,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available
0,Goa Express,http://goaexpress.com,Luxury Bus,AC,10:00 AM,8 hours,6:00 PM,4.5,1200,30
1,Kerala Express,http://keralaexpress.com,Economy Bus,Non-AC,11:00 AM,9 hours,8:00 PM,4.0,1000,25


In [64]:
df_westbengal[df_westbengal['Route_name']=='Barasat (West Bengal) to Kolaghat']

,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available


In [65]:
df_westbengal.to_csv('West_bengal.csv')

In [66]:
driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile");

time.sleep(5)

driver.maximize_window()

In [67]:
def route_link_name(path):
    route_names = []
    route_links = []
    wait = WebDriverWait(driver, 20)
    for page in range(1, 6):  # Looping through 5 pages

        # Find all <a> elements inside the <div class="route_details"> having link and Name
        ctu_link_elements = driver.find_elements(By.CSS_SELECTOR, path)

        #retrieve route links and name from the links
        for link in ctu_link_elements:
            href_value = link.get_attribute("href")
            link_text = link.text
            route_links.append(href_value)
            route_names.append(link_text)
            print(f"Link: {href_value}, Name: {link_text}")
        
        #navigate to next page with all the buttons clicked
        try:
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@class="DC_117_paginationTable"]')))
            next_page = driver.find_element(By.XPATH, f"//*[@id='root']/div/div[4]/div[12]/div[{page+1}]")
            time.sleep(2)
            next_page.click()
        except:
            print("No more pages to scroll")
            break
    return route_names,route_links

In [68]:
path = "div.route_details a.route"
Route_Names,Route_links = route_link_name(path)

Link: https://www.redbus.in/bus-tickets/durgapur-to-kolkata, Name: Durgapur (West Bengal) to Kolkata
Link: https://www.redbus.in/bus-tickets/kolkata-to-burdwan, Name: Kolkata to Burdwan
Link: https://www.redbus.in/bus-tickets/kolkata-to-durgapur, Name: Kolkata to Durgapur (West Bengal)
Link: https://www.redbus.in/bus-tickets/haldia-to-kolkata, Name: Haldia to Kolkata
Link: https://www.redbus.in/bus-tickets/kolkata-to-haldia, Name: Kolkata to Haldia
Link: https://www.redbus.in/bus-tickets/midnapore-to-kolkata, Name: Midnapore to Kolkata
Link: https://www.redbus.in/bus-tickets/kolkata-to-arambagh-west-bengal, Name: Kolkata to Arambagh (West Bengal)
Link: https://www.redbus.in/bus-tickets/kolkata-to-midnapore, Name: Kolkata to Midnapore
Link: https://www.redbus.in/bus-tickets/digha-to-kolkata, Name: Digha to Kolkata
Link: https://www.redbus.in/bus-tickets/kolkata-to-bankura, Name: Kolkata to Bankura
No more pages to scroll


In [69]:
dict = {'Route_name':Route_Names, 'Route_link':Route_links}
df = pd.DataFrame(dict)
# df.to_csv("southbengal_route_details.csv")
df.shape

(10, 2)

In [70]:
def scrolling_s():
    scrolling = True
    while scrolling:
        old_page_source = driver_s.page_source
        driver_s.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_page_source = driver_s.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

In [71]:
driver_s = webdriver.Chrome()
Route_name=[]
Route_link=[]
Bus_name=[]
Bus_type=[]
Depature_time =[]
Duration=[]
Reaching_time=[]
Rating=[]
Price=[]
Seat_Availability=[]
wait = WebDriverWait(driver_s, 30)

for ind,val in df.iterrows():
    route_name = val['Route_name']
    route_link = val['Route_link']
    
    driver_s.get(route_link)
    time.sleep(2)
    
    route_links = driver_s.find_elements(By.CSS_SELECTOR, "div.route_details a.route")
    
    for link in route_links:
        link.click()
        time.sleep(2)

    try:
        button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='button']")))
        button.click()
    except (NoSuchElementException, TimeoutException, NoSuchWindowException):
        print("Button not found, continuing with scraping...")
        
    time.sleep(4)

    scrolling_s()

    bus_name = driver_s.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_s.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    dept_time = driver_s.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
    dur = driver_s.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
    rea_time = driver_s.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
    try:
        rate=driver_s.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_s.find_elements(By.XPATH, "//div[@class='fare d-block']")
    seat_ava = driver_s.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")


    for name in bus_name:
        Route_name.append(route_name)
        Route_link.append(route_link)
        Bus_name.append(name.text)
    for ty in bus_type:
        Bus_type.append(ty.text)
    for dt in dept_time:
        Depature_time.append(dt.text)
    for d in dur:
        Duration.append(d.text)
    for rea in rea_time:
        Reaching_time.append(rea.text)
    for r in rate:
        Rating.append(r.text.split("\n")[0])
    for p in price:
        Price.append(p.text)
    for seat in seat_ava:
        Seat_Availability.append(seat.text)
print("Successfully scraped using Selenium")

Button not found, continuing with scraping...
Button not found, continuing with scraping...
Successfully scraped using Selenium


In [72]:
print(len(Route_name),len(Route_link),len(Bus_name),len(Bus_type),len(Depature_time),len(Duration),len(Reaching_time),len(Rating),len(Price),len(Seat_Availability))

119 119 119 119 119 119 119 0 119 119


In [74]:
Route_name = ['Goa Express', 'Kerala Express']
Route_link = ['http://goaexpress.com', 'http://keralaexpress.com']
Bus_name = ['Luxury Bus', 'Economy Bus']
Bus_type = ['AC', 'Non-AC']
Depature_time = ['10:00 AM', '11:00 AM']
Duration = ['8 hours', '9 hours']
Reaching_time = ['6:00 PM', '8:00 PM']
Rating = [4.5, 4.0]
Price = [1200, 1000]
Seat_Availability = [30, 25]

dict = {
    'Route_name': Route_name, 
    'Route_link': Route_link, 
    'Bus_name': Bus_name, 
    'Bus_type': Bus_type,
    'Depature_time': Depature_time, 
    'Duration': Duration, 
    'Reaching_time': Reaching_time, 
    'Rating': Rating, 
    'Price': Price, 
    'Seat_Available': Seat_Availability
}

# Create the DataFrame
import pandas as pd
df_southbengal = pd.DataFrame(dict)
df_southbengal


,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available
0,Goa Express,http://goaexpress.com,Luxury Bus,AC,10:00 AM,8 hours,6:00 PM,4.5,1200,30
1,Kerala Express,http://keralaexpress.com,Economy Bus,Non-AC,11:00 AM,9 hours,8:00 PM,4.0,1000,25


In [75]:
df_southbengal[df_southbengal['Route_name']=='Kolkata to Nandakumar (west bengal)']

,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available


In [76]:
df_southbengal.to_csv('South_Bengal.csv')

In [77]:
driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile");

time.sleep(5)

driver.maximize_window()

In [78]:
def route_link_name(path):
    route_names = []
    route_links = []
    wait = WebDriverWait(driver, 20)
    for page in range(1, 6):  # Looping through 5 pages

        # Find all <a> elements inside the <div class="route_details"> having link and Name
        pepsu_link_elements = driver.find_elements(By.CSS_SELECTOR, path)

        #retrieve route links and name from the links
        for link in pepsu_link_elements:
            href_value = link.get_attribute("href")
            link_text = link.text
            route_links.append(href_value)
            route_names.append(link_text)
            print(f"Link: {href_value}, Name: {link_text}")
        
        #navigate to next page with all the buttons clicked
        try:
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@class="DC_117_paginationTable"]')))
            next_page = driver.find_element(By.XPATH, f"//*[@id='root']/div/div[4]/div[12]/div[{page+1}]")
            time.sleep(2)
            next_page.click()
        except:
            print("No more pages to scroll")
            break
    return route_names,route_links

In [79]:
path = "div.route_details a.route"
Route_Names,Route_links = route_link_name(path)

No more pages to scroll


In [80]:
dict = {'Route_name':Route_Names, 'Route_link':Route_links}
df = pd.DataFrame(dict)
df.to_csv("rajasthan_route_details.csv")
df.shape

(0, 2)

In [81]:
def scrolling_r():
    scrolling = True
    while scrolling:
        old_page_source = driver_r.page_source
        driver_r.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(4)
        new_page_source = driver_r.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

In [82]:
driver_r = webdriver.Chrome()
Route_name=[]
Route_link=[]
Bus_name=[]
Bus_type=[]
Depature_time =[]
Duration=[]
Reaching_time=[]
Rating=[]
Price=[]
Seat_Availability=[]
wait = WebDriverWait(driver_r, 30)

for ind,val in df.iterrows():
    route_name = val['Route_name']
    route_link = val['Route_link']
    
    driver_r.get(route_link)
    time.sleep(2)
    
    route_links = driver_r.find_elements(By.CSS_SELECTOR, "div.route_details a.route")
    
    for link in route_links:
        link.click()
        time.sleep(2)

    try:
        button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='button']")))
        button.click()
    except (NoSuchElementException, TimeoutException, NoSuchWindowException):
        print("Button not found, continuing with scraping...")
        
    time.sleep(4)

    scrolling_r()

    bus_name = driver_r.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_r.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    dept_time = driver_r.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
    dur = driver_r.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
    rea_time = driver_r.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
    try:
        rate=driver_r.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_r.find_elements(By.XPATH, "//div[@class='fare d-block']")
    seat_ava = driver_r.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")


    for name in bus_name:
        Route_name.append(route_name)
        Route_link.append(route_link)
        Bus_name.append(name.text)
    for ty in bus_type:
        Bus_type.append(ty.text)
    for dt in dept_time:
        Depature_time.append(dt.text)
    for d in dur:
        Duration.append(d.text)
    for rea in rea_time:
        Reaching_time.append(rea.text)
    for r in rate:
        Rating.append(r.text.split("\n")[0])
    for p in price:
        Price.append(p.text)
    for seat in seat_ava:
        Seat_Availability.append(seat.text)
print("Successfully scraped using Selenium")

Successfully scraped using Selenium


In [83]:
print(len(Route_name),len(Route_link),len(Bus_name),len(Bus_type),len(Depature_time),len(Duration),len(Reaching_time),len(Rating),len(Price),len(Seat_Availability))

0 0 0 0 0 0 0 0 0 0


In [84]:
dict = {'Route_name':Route_name, 'Route_link':Route_link, 'Bus_name':Bus_name,'Bus_type':Bus_type,
                         'Depature_time':Depature_time, 'Duration':Duration, 'Reaching_time':Reaching_time,
                         'Rating':Rating, 'Price':Price, 'Seat_Available':Seat_Availability}
df_rajasthan = pd.DataFrame(dict)
df_rajasthan

,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available


In [85]:
df_rajasthan.to_csv('Rajasthan.csv')

In [86]:
driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile");

time.sleep(5)

driver.maximize_window()

In [87]:
def route_link_name(path):
    route_names = []
    route_links = []
    wait = WebDriverWait(driver, 20)
    for page in range(1, 6):  # Looping through 5 pages

        # Find all <a> elements inside the <div class="route_details"> having link and Name
        nbtc_link_elements = driver.find_elements(By.CSS_SELECTOR, path)

        #retrieve route links and name from the links
        for link in nbtc_link_elements:
            href_value = link.get_attribute("href")
            link_text = link.text
            route_links.append(href_value)
            route_names.append(link_text)
            print(f"Link: {href_value}, Name: {link_text}")
        
        #navigate to next page with all the buttons clicked
        try:
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@class="DC_117_paginationTable"]')))
            next_page = driver.find_element(By.XPATH, f"//*[@id='root']/div/div[4]/div[12]/div[{page+1}]")
            time.sleep(2)
            next_page.click()
        except:
            print("No more pages to scroll")
            break
    return route_names,route_links

In [88]:
path = "div.route_details a.route"
Route_Names,Route_links = route_link_name(path)

Link: https://www.redbus.in/bus-tickets/hyderabad-to-vijayawada, Name: Hyderabad to Vijayawada
Link: https://www.redbus.in/bus-tickets/hyderabad-to-khammam, Name: Hyderabad to Khammam
Link: https://www.redbus.in/bus-tickets/khammam-to-hyderabad, Name: Khammam to Hyderabad
Link: https://www.redbus.in/bus-tickets/hyderabad-to-srisailam, Name: Hyderabad to Srisailam
Link: https://www.redbus.in/bus-tickets/hyderabad-to-karimnagar, Name: Hyderabad to Karimnagar
Link: https://www.redbus.in/bus-tickets/hyderabad-to-mancherial, Name: Hyderabad to Mancherial
Link: https://www.redbus.in/bus-tickets/hyderabad-to-sathupalli, Name: Hyderabad to Sathupally
Link: https://www.redbus.in/bus-tickets/hyderabad-to-adilabad, Name: Hyderabad to Adilabad
Link: https://www.redbus.in/bus-tickets/hyderabad-to-nirmal, Name: Hyderabad to Nirmal
Link: https://www.redbus.in/bus-tickets/hyderabad-to-bhadrachalam, Name: Hyderabad to Bhadrachalam
No more pages to scroll


In [89]:
dict = {'Route_name':Route_Names, 'Route_link':Route_links}
df = pd.DataFrame(dict)
# df.to_csv("telangana_route_details.csv")
df.shape

(10, 2)

In [90]:
def scrolling_t():
    scrolling = True
    while scrolling:
        old_page_source = driver_t.page_source
        driver_t.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_page_source = driver_t.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

In [91]:
driver_t = webdriver.Chrome()
Route_name=[]
Route_link=[]
Bus_name=[]
Bus_type=[]
Depature_time =[]
Duration=[]
Reaching_time=[]
Rating=[]
Price=[]
Seat_Availability=[]
wait = WebDriverWait(driver_t, 30)

for ind,val in df.iterrows():
    route_name = val['Route_name']
    route_link = val['Route_link']
    
    driver_t.get(route_link)
    time.sleep(2)
    
    route_links = driver_t.find_elements(By.CSS_SELECTOR, "div.route_details a.route")
    
    for link in route_links:
        link.click()
        time.sleep(3)

    try:
        button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='button']")))
        button.click()
    except (NoSuchElementException, TimeoutException, NoSuchWindowException,ElementClickInterceptedException):
        print("Button not found, continuing with scraping...")
        
    time.sleep(4)

    scrolling_t()

    bus_name = driver_t.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_t.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    dept_time = driver_t.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
    dur = driver_t.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
    rea_time = driver_t.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
    try:
        rate=driver_t.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_t.find_elements(By.XPATH, "//div[@class='fare d-block']")
    seat_ava = driver_t.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")


    for name in bus_name:
        Route_name.append(route_name)
        Route_link.append(route_link)
        Bus_name.append(name.text)
    for ty in bus_type:
        Bus_type.append(ty.text)
    for dt in dept_time:
        Depature_time.append(dt.text)
    for d in dur:
        Duration.append(d.text)
    for rea in rea_time:
        Reaching_time.append(rea.text)
    for r in rate:
        Rating.append(r.text.split("\n")[0])
    for p in price:
        Price.append(p.text)
    for seat in seat_ava:
        Seat_Availability.append(seat.text)
print("Successfully scraped using Selenium")

Button not found, continuing with scraping...
Successfully scraped using Selenium


In [92]:
print(len(Route_name),len(Route_link),len(Bus_name),len(Bus_type),len(Depature_time),len(Duration),len(Reaching_time),len(Rating),len(Price),len(Seat_Availability))

84 84 84 84 84 84 84 0 84 84


In [97]:
import pandas as pd

Route_name = ['Route1', 'Route2']
Route_link = ['link1', 'link2', 'link3']  # This has 3 elements
Bus_name = ['Bus1', 'Bus2']
Bus_type = ['AC', 'Non-AC']

# Pad shorter lists with None
while len(Route_name) < len(Route_link):
    Route_name.append(None)

while len(Bus_name) < len(Route_link):
    Bus_name.append(None)

while len(Bus_type) < len(Route_link):
    Bus_type.append(None)

# Check the lengths
print(len(Route_name))  # All should now be the same length
print(len(Route_link))
print(len(Bus_name))
print(len(Bus_type))

# Create DataFrame
dict = {
    'Route_name': Route_name,
    'Route_link': Route_link,
    'Bus_name': Bus_name,
    'Bus_type': Bus_type,
}

df = pd.DataFrame(dict)
print(df)


3
3
3
3
  Route_name Route_link Bus_name Bus_type
0     Route1      link1     Bus1       AC
1     Route2      link2     Bus2   Non-AC
2       None      link3     None     None


In [178]:
df_telangana.to_csv('Telangana.csv')

In [100]:
driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile");

time.sleep(5)

driver.maximize_window()

In [101]:
def route_link_name(path):
    route_names = []
    route_links = []
    wait = WebDriverWait(driver, 20)
    for page in range(1, 6):  # Looping through 5 pages

        # Find all <a> elements inside the <div class="route_details"> having link and Name
        bsrtc_link_elements = driver.find_elements(By.CSS_SELECTOR, path)

        #retrieve route links and name from the links
        for link in bsrtc_link_elements:
            href_value = link.get_attribute("href")
            link_text = link.text
            route_links.append(href_value)
            route_names.append(link_text)
            print(f"Link: {href_value}, Name: {link_text}")
        
        #navigate to next page with all the buttons clicked
        try:
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@class="DC_117_paginationTable"]')))
            next_page = driver.find_element(By.XPATH, f"//*[@id='root']/div/div[4]/div[12]/div[{page+1}]")
            time.sleep(2)
            next_page.click()
        except:
            print("No more pages to scroll")
            break
    return route_names,route_links

In [102]:
path = "div.route_details a.route"
Route_Names,Route_links = route_link_name(path)

Link: https://www.redbus.in/bus-tickets/bareilly-to-delhi, Name: Bareilly (Uttar Pradesh) to Delhi
Link: https://www.redbus.in/bus-tickets/lucknow-to-allahabad, Name: Lucknow to Prayagraj(Uttar Pradesh)
Link: https://www.redbus.in/bus-tickets/allahabad-to-lucknow, Name: Prayagraj(Uttar Pradesh) to Lucknow
Link: https://www.redbus.in/bus-tickets/delhi-to-bareilly, Name: Delhi to Bareilly (Uttar Pradesh)
Link: https://www.redbus.in/bus-tickets/aligarh-uttar-pradesh-to-delhi, Name: Aligarh (uttar pradesh) to Delhi
Link: https://www.redbus.in/bus-tickets/lucknow-to-bareilly, Name: Lucknow to Bareilly (Uttar Pradesh)
Link: https://www.redbus.in/bus-tickets/lucknow-to-agra, Name: Lucknow to Agra
Link: https://www.redbus.in/bus-tickets/ayodhya-to-allahabad, Name: Ayodhya to Allahabad
Link: https://www.redbus.in/bus-tickets/allahabad-to-ayodhya, Name: Prayagraj(Uttar Pradesh) to Ayodhya
Link: https://www.redbus.in/bus-tickets/bareilly-to-lucknow, Name: Bareilly (Uttar Pradesh) to Lucknow
No mo

In [103]:
dict = {'Route_name':Route_Names, 'Route_link':Route_links}
df = pd.DataFrame(dict)
# df.to_csv("uttarpradesh_route_details.csv")
df.shape

(10, 2)

In [104]:
def scrolling_u():
    scrolling = True
    while scrolling:
        old_page_source = driver_u.page_source
        driver_u.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_page_source = driver_u.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

In [105]:
driver_u = webdriver.Chrome()
Route_name=[]
Route_link=[]
Bus_name=[]
Bus_type=[]
Depature_time =[]
Duration=[]
Reaching_time=[]
Rating=[]
Price=[]
Seat_Availability=[]
wait = WebDriverWait(driver_u, 30)

for ind,val in df.iterrows():
    route_name = val['Route_name']
    route_link = val['Route_link']
    
    driver_u.get(route_link)
    time.sleep(2)
    
    route_links = driver_u.find_elements(By.CSS_SELECTOR, "div.route_details a.route")
    
    for link in route_links:
        link.click()
        time.sleep(2)

    try:
        button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='button']")))
        button.click()
    except (NoSuchElementException, TimeoutException,NoSuchWindowException):
        print("Button not found, continuing with scraping...")
        
    time.sleep(4)

    scrolling_u()

    bus_name = driver_u.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_u.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    dept_time = driver_u.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
    dur = driver_u.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
    rea_time = driver_u.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
    try:
        rate=driver_u.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_u.find_elements(By.XPATH, "//div[@class='fare d-block']")
    seat_ava = driver_u.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")


    for name in bus_name:
        Route_name.append(route_name)
        Route_link.append(route_link)
        Bus_name.append(name.text)
    for ty in bus_type:
        Bus_type.append(ty.text)
    for dt in dept_time:
        Depature_time.append(dt.text)
    for d in dur:
        Duration.append(d.text)
    for rea in rea_time:
        Reaching_time.append(rea.text)
    for r in rate:
        Rating.append(r.text.split("\n")[0])
    for p in price:
        Price.append(p.text)
    for seat in seat_ava:
        Seat_Availability.append(seat.text)
print("Successfully scraped using Selenium")

Button not found, continuing with scraping...
Successfully scraped using Selenium


In [106]:
print(len(Route_name),len(Route_link),len(Bus_name),len(Bus_type),len(Depature_time),len(Duration),len(Reaching_time),len(Rating),len(Price),len(Seat_Availability))

168 168 168 168 168 168 168 0 168 168


In [112]:
import pandas as pd

# Sample lists (Replace with your actual lists)
Route_name = ['Route1', 'Route2', 'Route3']
Route_link = ['Link1', 'Link2']
Bus_name = ['Bus1', 'Bus2', 'Bus3']
Bus_type = ['Type1', 'Type2', 'Type3']
Depature_time = ['10:00 AM', '12:00 PM', '2:00 PM']
Duration = ['2h', '3h', '4h']
Reaching_time = ['12:00 PM', '3:00 PM', '6:00 PM']
Rating = [4.5, 4.2, 4.8]
Price = [100, 150, 200]
Seat_Availability = [20, 15, 10]

# Check the lengths of each list
print(f"Length of Route_name: {len(Route_name)}")
print(f"Length of Route_link: {len(Route_link)}")
print(f"Length of Bus_name: {len(Bus_name)}")
print(f"Length of Bus_type: {len(Bus_type)}")
print(f"Length of Depature_time: {len(Depature_time)}")
print(f"Length of Duration: {len(Duration)}")
print(f"Length of Reaching_time: {len(Reaching_time)}")
print(f"Length of Rating: {len(Rating)}")
print(f"Length of Price: {len(Price)}")
print(f"Length of Seat_Availability: {len(Seat_Availability)}")

# Find the maximum length
max_length = max(len(Route_name), len(Route_link), len(Bus_name), len(Bus_type), 
                 len(Depature_time), len(Duration), len(Reaching_time), 
                 len(Rating), len(Price), len(Seat_Availability))

# Pad shorter lists to make them all equal to max_length
def pad_list(lst, max_length):
    return lst + [None] * (max_length - len(lst))

Route_name = pad_list(Route_name, max_length)
Route_link = pad_list(Route_link, max_length)
Bus_name = pad_list(Bus_name, max_length)
Bus_type = pad_list(Bus_type, max_length)
Depature_time = pad_list(Depature_time, max_length)
Duration = pad_list(Duration, max_length)
Reaching_time = pad_list(Reaching_time, max_length)
Rating = pad_list(Rating, max_length)
Price = pad_list(Price, max_length)
Seat_Availability = pad_list(Seat_Availability, max_length)

# Now create the dictionary and DataFrame
dict = {
    'Route_name': Route_name,
    'Route_link': Route_link,
    'Bus_name': Bus_name,
    'Bus_type': Bus_type,
    'Depature_time': Depature_time,
    'Duration': Duration,
    'Reaching_time': Reaching_time,
    'Rating': Rating,
    'Price': Price,
    'Seat_Available': Seat_Availability
}

# Create DataFrame
df_uttarpradesh = pd.DataFrame(dict)
print(df_uttarpradesh)


Length of Route_name: 3
Length of Route_link: 2
Length of Bus_name: 3
Length of Bus_type: 3
Length of Depature_time: 3
Length of Duration: 3
Length of Reaching_time: 3
Length of Rating: 3
Length of Price: 3
Length of Seat_Availability: 3
  Route_name Route_link Bus_name Bus_type Depature_time Duration  \
0     Route1      Link1     Bus1    Type1      10:00 AM       2h   
1     Route2      Link2     Bus2    Type2      12:00 PM       3h   
2     Route3       None     Bus3    Type3       2:00 PM       4h   

  Reaching_time  Rating  Price  Seat_Available  
0      12:00 PM     4.5    100              20  
1       3:00 PM     4.2    150              15  
2       6:00 PM     4.8    200              10  


In [113]:
df_uttarpradesh.to_csv('UttarPradesh.csv')

In [114]:
driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile");

time.sleep(5)

driver.maximize_window()

In [115]:
def route_link_name(path):
    route_names = []
    route_links = []
    wait = WebDriverWait(driver, 20)
    for page in range(1, 6):  # Looping through 5 pages

        # Find all <a> elements inside the <div class="route_details"> having link and Name
        kaac_link_elements = driver.find_elements(By.CSS_SELECTOR, path)

        #retrieve route links and name from the links
        for link in kaac_link_elements:
            href_value = link.get_attribute("href")
            link_text = link.text
            route_links.append(href_value)
            route_names.append(link_text)
            print(f"Link: {href_value}, Name: {link_text}")
        
        #navigate to next page with all the buttons clicked
        try:
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@class="DC_117_paginationTable"]')))
            next_page = driver.find_element(By.XPATH, f"//*[@id='root']/div/div[4]/div[12]/div[{page+1}]")
            time.sleep(2)
            next_page.click()
        except:
            print("No more pages to scroll")
            break
    return route_names,route_links

In [116]:
kaac = "div.route_details a.route"
Route_Names,Route_links = route_link_name(kaac)

Link: https://www.redbus.in/bus-tickets/delhi-to-shimla, Name: Delhi to Shimla
Link: https://www.redbus.in/bus-tickets/shimla-to-delhi, Name: Shimla to Delhi
Link: https://www.redbus.in/bus-tickets/chandigarh-to-hamirpur-himachal-pradesh, Name: Chandigarh to Hamirpur (Himachal Pradesh)
Link: https://www.redbus.in/bus-tickets/hamirpur-himachal-pradesh-to-chandigarh, Name: Hamirpur (Himachal Pradesh) to Chandigarh
Link: https://www.redbus.in/bus-tickets/delhi-to-chandigarh, Name: Delhi to Chandigarh
Link: https://www.redbus.in/bus-tickets/manali-to-chandigarh, Name: Manali to Chandigarh
Link: https://www.redbus.in/bus-tickets/shimla-to-chandigarh, Name: Shimla to Chandigarh
Link: https://www.redbus.in/bus-tickets/chandigarh-to-manali, Name: Chandigarh to Manali
Link: https://www.redbus.in/bus-tickets/shimla-to-manali, Name: Shimla to Manali
Link: https://www.redbus.in/bus-tickets/dharamshala-to-chandigarh, Name: Dharamshala (Himachal Pradesh) to Chandigarh
No more pages to scroll


In [117]:
dict = {'Route_name':Route_Names, 'Route_link':Route_links}
df = pd.DataFrame(dict)
df.to_csv("himachalpradesh_route_details.csv")
df.shape

(10, 2)

In [118]:
def scrolling_hp():
    scrolling = True
    while scrolling:
        old_page_source = driver_hp.page_source
        driver_hp.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        new_page_source = driver_hp.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

In [119]:
driver_hp = webdriver.Chrome()
Route_name=[]
Route_link=[]
Bus_name=[]
Bus_type=[]
Depature_time =[]
Duration=[]
Reaching_time=[]
Rating=[]
Price=[]
Seat_Availability=[]
wait = WebDriverWait(driver_hp, 30)

for ind,val in df.iterrows():
    route_name = val['Route_name']
    route_link = val['Route_link']
    
    driver_hp.get(route_link)
    time.sleep(2)
    
    route_links = driver_hp.find_elements(By.CSS_SELECTOR, "div.route_details a.route")
    
    for link in route_links:
        link.click()
        time.sleep(2)

    try:
        button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='button']")))
        button.click()
    except (NoSuchElementException, TimeoutException):
        print("Button not found, continuing with scraping...")
    
    time.sleep(4)
    
    scrolling_hp()
    
    bus_name = driver_hp.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_hp.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    dept_time = driver_hp.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
    dur = driver_hp.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
    rea_time = driver_hp.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
    try:
        rate=driver_hp.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_hp.find_elements(By.XPATH, "//div[@class='fare d-block']")
    seat_ava = driver_hp.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")


    for name in bus_name:
        Route_name.append(route_name)
        Route_link.append(route_link)
        Bus_name.append(name.text)
    for ty in bus_type:
        Bus_type.append(ty.text)
    for dt in dept_time:
        Depature_time.append(dt.text)
    for d in dur:
        Duration.append(d.text)
    for rea in rea_time:
        Reaching_time.append(rea.text)
    for r in rate:
        Rating.append(r.text.split("\n")[0])
    for p in price:
        Price.append(p.text)
    for seat in seat_ava:
        Seat_Availability.append(seat.text)
print("Successfully scraped using Selenium")

Button not found, continuing with scraping...
Button not found, continuing with scraping...
Successfully scraped using Selenium


In [120]:
print(len(Route_name),len(Route_link),len(Bus_name),len(Bus_type),len(Depature_time),len(Duration),len(Reaching_time),len(Rating),len(Price),len(Seat_Availability))

79 79 79 79 79 79 79 0 79 79


In [133]:
# Sample lists with lengths to check
Route_name = ["Route1", "Route2", "Route3"]
Route_link = ["Link1", "Link2"]
Bus_name = ["BusA", "BusB", "BusC"]
Bus_type = ["AC", "Non-AC", "AC"]
Depature_time = ["10:00", "11:00", "12:00"]
Duration = [5, 6, 7]
Reaching_time = ["15:00", "16:00", "17:00"]
Rating = [4.5, 4.2, 4.8]
Price = [500, 600, 550]
Seat_Availability = [20, 18, 15]

# Check the lengths of each list
print(f"Route_name length: {len(Route_name)}")
print(f"Route_link length: {len(Route_link)}")
print(f"Bus_name length: {len(Bus_name)}")
print(f"Bus_type length: {len(Bus_type)}")
print(f"Depature_time length: {len(Depature_time)}")
print(f"Duration length: {len(Duration)}")
print(f"Reaching_time length: {len(Reaching_time)}")
print(f"Rating length: {len(Rating)}")
print(f"Price length: {len(Price)}")
print(f"Seat_Availability length: {len(Seat_Availability)}")

# Adjust lists to be the same length (pad shorter lists with None or NaN)
# Here, Route_link is shorter, so we will pad it with None values.

max_len = max(len(Route_name), len(Route_link), len(Bus_name), len(Bus_type), len(Depature_time), 
              len(Duration), len(Reaching_time), len(Rating), len(Price), len(Seat_Availability))

# Pad lists with None to make them the same length
Route_link.extend([None] * (max_len - len(Route_link)))
Bus_name.extend([None] * (max_len - len(Bus_name)))
Bus_type.extend([None] * (max_len - len(Bus_type)))
Depature_time.extend([None] * (max_len - len(Depature_time)))
Duration.extend([None] * (max_len - len(Duration)))
Reaching_time.extend([None] * (max_len - len(Reaching_time)))
Rating.extend([None] * (max_len - len(Rating)))
Price.extend([None] * (max_len - len(Price)))
Seat_Availability.extend([None] * (max_len - len(Seat_Availability)))

# Now create the DataFrame
dict_data = {
    'Route_name': Route_name,
    'Route_link': Route_link,
    'Bus_name': Bus_name,
    'Bus_type': Bus_type,
    'Depature_time': Depature_time,
    'Duration': Duration,
    'Reaching_time': Reaching_time,
    'Rating': Rating,
    'Price': Price,
    'Seat_Available': Seat_Availability
}

# Create the DataFrame
df_himachalpradesh = pd.DataFrame(dict_data)

# Display the DataFrame
print(df_himachalpradesh)


Route_name length: 3
Route_link length: 2
Bus_name length: 3
Bus_type length: 3
Depature_time length: 3
Duration length: 3
Reaching_time length: 3
Rating length: 3
Price length: 3
Seat_Availability length: 3
  Route_name Route_link Bus_name Bus_type Depature_time  Duration  \
0     Route1      Link1     BusA       AC         10:00         5   
1     Route2      Link2     BusB   Non-AC         11:00         6   
2     Route3       None     BusC       AC         12:00         7   

  Reaching_time  Rating  Price  Seat_Available  
0         15:00     4.5    500              20  
1         16:00     4.2    600              18  
2         17:00     4.8    550              15  


In [134]:
df_himachalpradesh.to_csv('HimachalPradesh.csv')

In [137]:
driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/jksrtc");

time.sleep(5)

driver.maximize_window()

In [136]:
def route_link_name(path):
    route_names = []
    route_links = []
    wait = WebDriverWait(driver, 20)
    for page in range(1, 6):  # Looping through 5 pages

        # Find all <a> elements inside the <div class="route_details"> having link and Name
        jkrtc_link_elements = driver.find_elements(By.CSS_SELECTOR, path)

        #retrieve route links and name from the links
        for link in jkrtc_link_elements:
            href_value = link.get_attribute("href")
            link_text = link.text
            route_links.append(href_value)
            route_names.append(link_text)
            print(f"Link: {href_value}, Name: {link_text}")
        
        #navigate to next page with all the buttons clicked
        try:
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@class="DC_117_paginationTable"]')))
            next_page = driver.find_element(By.XPATH, f"//*[@id='root']/div/div[4]/div[12]/div[{page+1}]")
            time.sleep(2)
            next_page.click()
        except:
            print("No more pages to scroll")
            break
    return route_names,route_links

In [138]:
page = "div.route_details a.route"
Route_Names,Route_links = route_link_name(page)

Link: https://www.redbus.in/bus-tickets/haridwar-to-jammu, Name: Haridwar to Jammu (j and k)
Link: https://www.redbus.in/bus-tickets/katra-to-jammu, Name: Katra (jammu and kashmir) to Jammu (j and k)
Link: https://www.redbus.in/bus-tickets/jammu-to-haridwar, Name: Jammu (j and k) to Haridwar
Link: https://www.redbus.in/bus-tickets/poonch-to-jammu, Name: Poonch to Jammu (j and k)
Link: https://www.redbus.in/bus-tickets/jammu-to-mendhar-j-k, Name: Jammu (j and k) to Mendhar (J & K)
Link: https://www.redbus.in/bus-tickets/jammu-to-poonch, Name: Jammu (j and k) to Poonch
Link: https://www.redbus.in/bus-tickets/jammu-to-kishtwar, Name: Jammu (j and k) to Kishtwar
Link: https://www.redbus.in/bus-tickets/kishtwar-to-jammu, Name: Kishtwar to Jammu (j and k)
No more pages to scroll


In [139]:
dict = {'Route_name':Route_Names, 'Route_link':Route_links}
df = pd.DataFrame(dict)
# df.to_csv("jammu_route_details.csv")
df.shape

(8, 2)

In [140]:
def scrolling_j():
    scrolling = True
    while scrolling:
        old_page_source = driver_j.page_source
        driver_j.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(4)
        new_page_source = driver_j.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

In [141]:
driver_j = webdriver.Chrome()
Route_name=[]
Route_link=[]
Bus_name=[]
Bus_type=[]
Depature_time =[]
Duration=[]
Reaching_time=[]
Rating=[]
Price=[]
Seat_Availability=[]
wait = WebDriverWait(driver_j, 30)

for ind,val in df.iterrows():
    route_name = val['Route_name']
    route_link = val['Route_link']
    
    driver_j.get(route_link)
    time.sleep(2)
    
    route_links = driver_j.find_elements(By.CSS_SELECTOR, "div.route_details a.route")
    
    for link in route_links:
        link.click()
        time.sleep(2)

    try:
        button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='button']")))
        button.click()
    except (NoSuchElementException, TimeoutException, NoSuchWindowException):
        print("Button not found, continuing with scraping...")
    
    time.sleep(4)

    scrolling_j()

    bus_name = driver_j.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_j.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    dept_time = driver_j.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
    dur = driver_j.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
    rea_time = driver_j.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
    try:
        rate=driver_j.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_j.find_elements(By.XPATH, "//div[@class='fare d-block']")
    seat_ava = driver_j.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")


    for name in bus_name:
        Route_name.append(route_name)
        Route_link.append(route_link)
        Bus_name.append(name.text)
    for ty in bus_type:
        Bus_type.append(ty.text)
    for dt in dept_time:
        Depature_time.append(dt.text)
    for d in dur:
        Duration.append(d.text)
    for rea in rea_time:
        Reaching_time.append(rea.text)
    for r in rate:
        Rating.append(r.text.split("\n")[0])
    for p in price:
        Price.append(p.text)
    for seat in seat_ava:
        Seat_Availability.append(seat.text)
print("Successfully scraped using Selenium")

Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Button not found, continuing with scraping...
Successfully scraped using Selenium


In [142]:
print(len(Route_name),len(Route_link),len(Bus_name),len(Bus_type),len(Depature_time),len(Duration),len(Reaching_time),len(Rating),len(Price),len(Seat_Availability))

15 15 15 15 15 15 15 0 15 15


In [144]:
import pandas as pd

# Sample lists (replace with your actual data)
Route_name = ["Route1", "Route2", "Route3"]
Route_link = ["Link1", "Link2"]
Bus_name = ["BusA", "BusB", "BusC"]
Bus_type = ["AC", "Non-AC", "AC"]
Depature_time = ["10:00", "11:00", "12:00"]
Duration = [5, 6, 7]
Reaching_time = ["15:00", "16:00", "17:00"]
Rating = [4.5, 4.2, 4.8]
Price = [500, 600, 550]
Seat_Availability = [20, 18, 15]

# Check the lengths of each list
print(f"Route_name length: {len(Route_name)}")
print(f"Route_link length: {len(Route_link)}")
print(f"Bus_name length: {len(Bus_name)}")
print(f"Bus_type length: {len(Bus_type)}")
print(f"Depature_time length: {len(Depature_time)}")
print(f"Duration length: {len(Duration)}")
print(f"Reaching_time length: {len(Reaching_time)}")
print(f"Rating length: {len(Rating)}")
print(f"Price length: {len(Price)}")
print(f"Seat_Availability length: {len(Seat_Availability)}")

# Find the maximum length
max_len = max(len(Route_name), len(Route_link), len(Bus_name), len(Bus_type), len(Depature_time), 
              len(Duration), len(Reaching_time), len(Rating), len(Price), len(Seat_Availability))

# Pad shorter lists with None to make them all the same length
Route_link.extend([None] * (max_len - len(Route_link)))
Bus_name.extend([None] * (max_len - len(Bus_name)))
Bus_type.extend([None] * (max_len - len(Bus_type)))
Depature_time.extend([None] * (max_len - len(Depature_time)))
Duration.extend([None] * (max_len - len(Duration)))
Reaching_time.extend([None] * (max_len - len(Reaching_time)))
Rating.extend([None] * (max_len - len(Rating)))
Price.extend([None] * (max_len - len(Price)))
Seat_Availability.extend([None] * (max_len - len(Seat_Availability)))

# Now create the DataFrame
dict_data = {
    'Route_name': Route_name,
    'Route_link': Route_link,
    'Bus_name': Bus_name,
    'Bus_type': Bus_type,
    'Depature_time': Depature_time,
    'Duration': Duration,
    'Reaching_time': Reaching_time,
    'Rating': Rating,
    'Price': Price,
    'Seat_Available': Seat_Availability
}

# Create the DataFrame
df_jammu = pd.DataFrame(dict_data)

# Display the DataFrame
print(df_jammu)


Route_name length: 3
Route_link length: 2
Bus_name length: 3
Bus_type length: 3
Depature_time length: 3
Duration length: 3
Reaching_time length: 3
Rating length: 3
Price length: 3
Seat_Availability length: 3
  Route_name Route_link Bus_name Bus_type Depature_time  Duration  \
0     Route1      Link1     BusA       AC         10:00         5   
1     Route2      Link2     BusB   Non-AC         11:00         6   
2     Route3       None     BusC       AC         12:00         7   

  Reaching_time  Rating  Price  Seat_Available  
0         15:00     4.5    500              20  
1         16:00     4.2    600              18  
2         17:00     4.8    550              15  


In [145]:
df_jammu[df_jammu['Route_name']=='Jammu (j and k) to Chandigarh']

,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available


In [146]:
df_jammu.to_csv('Jammu.csv') 

In [149]:
# Ensure df_telangana is correctly defined before using it in pd.concat
# Check the df_telangana DataFrame initialization
# Example: 
df_telangana = pd.DataFrame({
    # Add columns for df_telangana as needed
})

# Then concatenate the DataFrames
Red_bus_df = pd.concat([df_andhra, df_kerala, df_goa, df_westbengal, df_southbengal, df_rajasthan, 
                        df_telangana, df_uttarpradesh, df_himachalpradesh, df_jammu], ignore_index=True)

print(Red_bus_df)


        Route_name                Route_link     Bus_name Bus_type  \
0      Goa Express     http://goaexpress.com   Luxury Bus       AC   
1      Goa Express     http://goaexpress.com   Luxury Bus       AC   
2   Kerala Express  http://keralaexpress.com  Economy Bus   Non-AC   
3      Goa Express     http://goaexpress.com   Luxury Bus       AC   
4      Goa Express     http://goaexpress.com   Luxury Bus       AC   
5   Kerala Express  http://keralaexpress.com  Economy Bus   Non-AC   
6      Goa Express     http://goaexpress.com   Luxury Bus       AC   
7   Kerala Express  http://keralaexpress.com  Economy Bus   Non-AC   
8           Route1                     Link1         Bus1    Type1   
9           Route2                     Link2         Bus2    Type2   
10          Route3                      None         Bus3    Type3   
11          Route1                     Link1         BusA       AC   
12          Route2                     Link2         BusB   Non-AC   
13          Route3  

In [175]:
Red_bus_df = pd.concat([df_andhra,df_kerala,df_goa,df_westbengal,df_southbengal,df_rajasthan,df_telangana,
                      df_uttarpradesh,df_himachalpradesh,df_jammu],ignore_index=True)

In [183]:
import os

file_path = r"C:/Users/nagar/Documents/Guvi Projects/Red_bus_project_files/Red_bus_df.csv"

if os.path.exists(file_path):
    Red_bus_df = pd.read_csv(file_path)
    print("File loaded successfully.")
else:
    print(f"File not found at: {file_path}")


File not found at: C:/Users/nagar/Documents/Guvi Projects/Red_bus_project_files/Red_bus_df.csv


In [184]:
Red_bus_df

,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available
0,Goa Express,http://goaexpress.com,Luxury Bus,AC,10:00 AM,8 hours,6:00 PM,4.5,1200.0,30.0
1,Goa Express,http://goaexpress.com,Luxury Bus,AC,10:00 AM,8 hours,6:00 PM,4.5,1200.0,30.0
2,Kerala Express,http://keralaexpress.com,Economy Bus,Non-AC,11:00 AM,9 hours,8:00 PM,4.0,1000.0,20.0
3,Goa Express,http://goaexpress.com,Luxury Bus,AC,10:00 AM,8 hours,6:00 PM,4.5,1200.0,30.0
4,Goa Express,http://goaexpress.com,Luxury Bus,AC,10:00 AM,8 hours,6:00 PM,4.5,1200.0,30.0
5,Kerala Express,http://keralaexpress.com,Economy Bus,Non-AC,11:00 AM,9 hours,8:00 PM,4.0,1000.0,25.0
6,Goa Express,http://goaexpress.com,Luxury Bus,AC,10:00 AM,8 hours,6:00 PM,4.5,1200.0,30.0
7,Kerala Express,http://keralaexpress.com,Economy Bus,Non-AC,11:00 AM,9 hours,8:00 PM,4.0,1000.0,25.0
8,Route1,Link1,Bus1,Type1,10:00 AM,2h,12:00 PM,4.5,100.0,20.0
9,Route2,Link2,Bus2,Type2,12:00 PM,3h,3:00 PM,4.2,150.0,15.0


In [185]:
Red_bus_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Route_name      17 non-null     object 
 1   Route_link      14 non-null     object 
 2   Bus_name        17 non-null     object 
 3   Bus_type        17 non-null     object 
 4   Depature_time   17 non-null     object 
 5   Duration        17 non-null     object 
 6   Reaching_time   17 non-null     object 
 7   Rating          17 non-null     float64
 8   Price           17 non-null     float64
 9   Seat_Available  17 non-null     float64
dtypes: float64(3), object(7)
memory usage: 1.5+ KB


In [186]:
Red_bus_df[Red_bus_df['Rating']=='New']

,Route_name,Route_link,Bus_name,Bus_type,Depature_time,Duration,Reaching_time,Rating,Price,Seat_Available


In [187]:
Red_bus_df['Rating'] = pd.to_numeric(Red_bus_df['Rating'],errors='coerce')
rating_mean = round(Red_bus_df['Rating'].mean(),1)
Red_bus_df['Rating'].fillna(rating_mean,inplace=True)
Red_bus_df['Rating'] = Red_bus_df['Rating'].astype(float)

C:\Users\Dinesh K\AppData\Local\Temp\ipykernel_15660\3877615751.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  Red_bus_df['Rating'].fillna(rating_mean,inplace=True)


In [188]:
Red_bus_df.dtypes

Route_name         object
Route_link         object
Bus_name           object
Bus_type           object
Depature_time      object
Duration           object
Reaching_time      object
Rating            float64
Price             float64
Seat_Available    float64
dtype: object

In [192]:
# Ensure the 'Seat_Available' column is of string type
Red_bus_df['Seat_Available'] = Red_bus_df['Seat_Available'].astype(str)

# Perform the string replacement
Red_bus_df['Seat_Available'] = Red_bus_df['Seat_Available'].str.replace(r' Seats? available', '', regex=True)

# Convert the column to float first (to handle decimal numbers) and then to integer
Red_bus_df['Seat_Available'] = Red_bus_df['Seat_Available'].astype(float).astype(int)


In [193]:
Red_bus_df.dtypes

Route_name         object
Route_link         object
Bus_name           object
Bus_type           object
Depature_time      object
Duration           object
Reaching_time      object
Rating            float64
Price             float64
Seat_Available      int32
dtype: object

In [2]:
import os

file_path = "C:/Users/nagar/Documents/Guvi Projects/Red_bus_project_files/Red_bus_df.csv"
if os.path.exists(file_path):
    Red_bus_df = pd.read_csv(file_path, index_col=0)
else:
    print("File not found!")


File not found!


In [ ]:
Red_bus_df.to_csv('Red_bus_df.csv')